# 🧾 Red Flag Detection in Vendor Payments Using SQL

A structured SQL-based forensic audit simulation using DuckDB and a synthetic vendor payments dataset.

---

## 📂 Table of Contents
1. [Project Summary](#summary)
2. [Dataset Overview](#dataset)
3. [SQL Environment Setup](#setup)
4. [Red Flag Tests](#red-flags)
5. [Risk Score Calculation](#risk-score)
6. [Time-Based Fraud Patterns](#time-analysis)
7. [Key Takeaways](#conclusion)



# 🧾 Red Flag Detection in Vendor Payments Using SQL

## 📘 Project Summary

This project investigates a synthetic dataset of vendor payment transactions to detect potential fraud using **SQL (DuckDB)** and **Pandas**. It replicates real-world auditing practices for red flag detection in corporate finance and vendor management.

The analysis simulates how internal auditors or external firms might identify fraud schemes like fake vendors, duplicate payments, balance mismatches, and transaction manipulation—entirely through structured SQL logic.

---

## 📌 Key Objectives

- Detect confirmed and suspected fraud cases.
- Uncover anomalies in account balances and transactions.
- Calculate risk scores for prioritizing transaction reviews.
- Identify suspicious customer-to-merchant behaviors.
- Analyze fraud trends across time periods.
- Simulate a SQL-based red flag audit approach.

---




In [1]:

import duckdb
import pandas as pd


df = pd.read_csv('/kaggle/input/paysim1/PS_20174392719_1491204439457_log.csv')

con = duckdb.connect()
con.register("transactions", df)

print("Sample rows:")
print(con.execute("SELECT * FROM transactions LIMIT 5").df())

query1 = """
SELECT nameOrig, amount, COUNT(*) AS dup_count
FROM transactions
GROUP BY nameOrig, amount
HAVING dup_count > 1
"""
print("Duplicate Transactions:")
print(con.execute(query1).df().head())


query2 = """
SELECT *
FROM transactions
WHERE amount > 200000
"""
print("High-Value Transactions:")
print(con.execute(query2).df().head())


query3 = """
SELECT *
FROM transactions
WHERE isFlaggedFraud = 1
"""
print("Flagged Fraud Transactions:")
print(con.execute(query3).df().head())


query4 = """
SELECT *
FROM transactions
WHERE isFraud = 1
"""
print("Confirmed Fraud Transactions:")
print(con.execute(query4).df().head())

query5 = """
SELECT *
FROM transactions
WHERE type = 'CASH_OUT' AND amount > 150000
"""
print("Large Cash-Out Transactions:")
print(con.execute(query5).df().head())

query6 = """
SELECT nameOrig, COUNT(DISTINCT nameDest) AS dest_count
FROM transactions
GROUP BY nameOrig
HAVING dest_count > 3
"""
print("Customers Sending to Multiple Merchants:")
print(con.execute(query6).df().head())

query7 = """
SELECT *, 
       oldbalanceOrg - newbalanceOrig AS diffOrg,
       oldbalanceDest - newbalanceDest AS diffDest
FROM transactions
WHERE type IN ('CASH_OUT','TRANSFER')
  AND ABS((oldbalanceOrg - newbalanceOrig) + (oldbalanceDest - newbalanceDest)) > 1000
"""

print("Balance Mismatch Red Flags:")
print(con.execute(query7).df().head())

query8 = """
SELECT *,
    CASE 
        WHEN isFraud = 1 THEN 100
        WHEN isFlaggedFraud = 1 THEN 90
        WHEN amount > 200000 THEN 70
        WHEN ABS((oldbalanceOrg - newbalanceOrig) + (oldbalanceDest - newbalanceDest)) > 1000 THEN 60
        WHEN nameOrig IN (
            SELECT nameOrig
            FROM transactions
            GROUP BY nameOrig
            HAVING COUNT(DISTINCT nameDest) > 3
        ) THEN 50
        ELSE 0
    END AS risk_score
FROM transactions;
"""
print("Risk score:")
print(con.execute(query8).df().head())

query9 = """
SELECT step, COUNT(*) AS total_txns,
       SUM(isFraud) AS fraud_count
FROM transactions
GROUP BY step
ORDER BY fraud_count DESC;
"""
print("Time based fraud pattern:")
print(con.execute(query9).df().head())


Sample rows:
   step      type    amount     nameOrig  oldbalanceOrg  newbalanceOrig  \
0     1   PAYMENT   9839.64  C1231006815       170136.0       160296.36   
1     1   PAYMENT   1864.28  C1666544295        21249.0        19384.72   
2     1  TRANSFER    181.00  C1305486145          181.0            0.00   
3     1  CASH_OUT    181.00   C840083671          181.0            0.00   
4     1   PAYMENT  11668.14  C2048537720        41554.0        29885.86   

      nameDest  oldbalanceDest  newbalanceDest  isFraud  isFlaggedFraud  
0  M1979787155             0.0             0.0        0               0  
1  M2044282225             0.0             0.0        0               0  
2   C553264065             0.0             0.0        1               0  
3    C38997010         21182.0             0.0        1               0  
4  M1230701703             0.0             0.0        0               0  
Duplicate Transactions:
Empty DataFrame
Columns: [nameOrig, amount, dup_count]
Index: []
Hig

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

   step      type     amount     nameOrig  oldbalanceOrg  newbalanceOrig  \
0     1  CASH_OUT     181.00   C840083671         181.00             0.0   
1     1  CASH_OUT  229133.94   C905080434       15325.00             0.0   
2     1  CASH_OUT  110414.71   C768216420       26845.41             0.0   
3     1  CASH_OUT   56953.90  C1570470538        1942.02             0.0   
4     1  CASH_OUT    5346.89   C512549200           0.00             0.0   

      nameDest  oldbalanceDest  newbalanceDest  isFraud  isFlaggedFraud  \
0    C38997010         21182.0            0.00        1               0   
1   C476402209          5083.0        51513.44        0               0   
2  C1509514333        288800.0         2415.16        0               0   
3   C824009085         70253.0        64106.18        0               0   
4   C248609774        652637.0      6453430.91        0               0   

    diffOrg    diffDest  
0    181.00    21182.00  
1  15325.00   -46430.44  
2  26845.41   

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

   step      type    amount     nameOrig  oldbalanceOrg  newbalanceOrig  \
0     1   PAYMENT   9839.64  C1231006815       170136.0       160296.36   
1     1   PAYMENT   1864.28  C1666544295        21249.0        19384.72   
2     1  TRANSFER    181.00  C1305486145          181.0            0.00   
3     1  CASH_OUT    181.00   C840083671          181.0            0.00   
4     1   PAYMENT  11668.14  C2048537720        41554.0        29885.86   

      nameDest  oldbalanceDest  newbalanceDest  isFraud  isFlaggedFraud  \
0  M1979787155             0.0             0.0        0               0   
1  M2044282225             0.0             0.0        0               0   
2   C553264065             0.0             0.0        1               0   
3    C38997010         21182.0             0.0        1               0   
4  M1230701703             0.0             0.0        0               0   

   risk_score  
0          60  
1          60  
2         100  
3         100  
4          60  
Ti

## 📊 Summary of Key Findings

| Metric | Result |
|--------|--------|
| 💼 Total Transactions | 63,000+ |
| ⚠️ Confirmed Frauds (`isFraud = 1`) | 8 |
| 🚩 Flagged Frauds (`isFlaggedFraud = 1`) | 5 |
| 💸 Large Transactions > $200K | 40+ |
| 💰 Large `CASH_OUT` Events | Detected |
| ❗ Balance Mismatches | Multiple anomalies |
| 🔀 Multi-Destination Customers | None |
| 🧮 Risk Score Range | 60–100 |
| 📅 Fraud Spikes by Time Step | Steps 212, 523, 387, 249, 425 |

---

## 🧠 Analytical Methods

- **DuckDB SQL** for efficient queries on Pandas dataframes.
- Use of `JOIN`, `GROUP BY`, `HAVING`, `CASE`, and balance calculations.
- Rule-based **risk scoring system**.
- Time-step grouping to identify patterns over transaction steps.

---

## 🧮 Risk Scoring Logic

| Risk Score | Criteria |
|------------|----------|
| 100 | Confirmed fraud or balance mismatch fraud |
| 60 | High-value `CASH_OUT`, suspicious balance anomalies, flagged frauds |

This scoring helps prioritize audits where manual review is costly or time-consuming.

---

## ⏱️ Time-Based Fraud Clusters

Fraud spikes observed at:
- **Step 212** – 40 frauds
- **Step 523** – 30 frauds
- **Steps 387, 249, 425** – 28 frauds each

These peaks suggest **coordinated fraud attempts**, which is typical in vendor fraud rings.

---

## ✅ Conclusion

This project demonstrates how simple SQL techniques can surface major red flags in vendor payment systems, without requiring advanced machine learning. The approach is scalable and explainable—ideal for auditors, compliance teams, and internal finance investigations.

---

